# **Are You the Asshole? Judging r/AITA Drama with NLP and ML**
*Final Project for CMSC176: Natural Language Processing* </br> </br>
**Author**: Daenielle Rai Peladas </br>
**Last Modified**: December 8, 2024

In [1]:
!git clone https://github.com/omoraisu/AITAH-FAFO.git

Cloning into 'AITAH-FAFO'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (1141/1141), done.
remote: Compressing objects: 100% (1089/1089), done.
remote: Total 1141 (delta 490), reused 219 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (1141/1141), 9.87 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (490/490), done.


## **Data Acquisition**

In [2]:
import os
import re
import sys
import json
import pandas as pd

In [3]:
# Retrieve comment directory
comments_dir = os.path.join('AITAH-FAFO/scrapes/all-comments')

This function generates a DataFrame from all the scraped comments stored in the specified directory.

In [43]:
def create_dataframe_from_comments(dir_path):
    # Directory path where the JSON files are stored
    comments_dir = os.path.join('AITAH-FAFO/scrapes/all-comments')

    # Initialize a list to hold the data
    data = []

    # Go through all files in the directory
    for filename in os.listdir(comments_dir):
        if filename.endswith('.json'):
            file_path = os.path.join(comments_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                # Load the JSON file
                content = json.load(file)

                metadata = content['data']['submission_metadata']
                comments = content['data']['comments']

                # Extract the required fields
                title = metadata['title']
                selftext = metadata['selftext']
                aitah_tag = count_YTA_NTA(comments)

                # Append to the data list
                data.append({'title': title, 'selftext': selftext, 'aitah_tag': aitah_tag})
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    return df

This function analyzes the comment section to count occurrences of keywords related to 'You're the Asshole' (YTA) and 'Not the Asshole' (NTA). Based on the keyword counts, it determines the overall judgment of whether the subject is considered the asshole or not.

In [42]:
def count_YTA_NTA(comments_list):
    # Extract 'body' content from each comment
    bodies = [comment['body'] for comment in comments_list if 'body' in comment]

    # Initialize counters for YTA and NTA
    YTA = 0
    NTA = 0

    # Loop through each comment body and count occurrences of YTA and NTA (including YTAH and NTAH)
    for body in bodies:
        YTA += len(re.findall(r'\bYTA\b', body)) + len(re.findall(r'\bYTAH\b', body))
        NTA += len(re.findall(r'\bNTA\b', body)) + len(re.findall(r'\bNTAH\b', body))

    if YTA > NTA:
        return 1
    return 0

In [44]:
# Specify the root directory
root_directory = 'AITAH-FAFO'
df = create_dataframe_from_comments(root_directory)

In [45]:
df.head()

,title,selftext,aitah_tag
0,AITA for cutting off a friend for being distant?,"\n\nso ill set the scene, i, (16M) and my two ...",0
1,AITAH for being pissed at my mom,So I (14M) have been playing guitar for 5ish y...,0
2,AITAH for making a girl cry at work,I work in the legal department at a corporate ...,0
3,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0
4,AITAH for refusing to come home for Christmas ...,[deleted],0


## **Basic Insights**

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      574 non-null    object
 1   selftext   574 non-null    object
 2   aitah_tag  574 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.6+ KB


## **Data Wrangling**

In [47]:
df.isnull().sum()

,0
title,0
selftext,0
aitah_tag,0


There are no null values shown here. However, there were some posts that were deleted and therefore must be removed since the text in the post itself is necessary for analysis.

In [48]:
df = df[df['selftext'] != '[deleted]']
df = df[df['selftext'] != '[removed]']

In [49]:
pd.set_option('display.max_rows', None)
df.head(100)

,title,selftext,aitah_tag
0,AITA for cutting off a friend for being distant?,"\n\nso ill set the scene, i, (16M) and my two ...",0
1,AITAH for being pissed at my mom,So I (14M) have been playing guitar for 5ish y...,0
2,AITAH for making a girl cry at work,I work in the legal department at a corporate ...,0
3,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0
5,AITAH for asking my friend to do some work for...,Im an artist (30/m) and im currently working h...,0
6,AITAH for Bvtching about my Ex-Crush to his Ex?,"I’m a 24-year-old female living in India, and ...",0
7,AITA for refusing to stay pass my scheduled le...,I'm a daycare teacher and my shift usually sta...,0
8,AITA for looking better than the bride,Ok so I 17f was invited to my BFFs sisters wed...,1
9,AITAH For not letting my Brother In Law hunt o...,My SIL and BIL (Wife's sister and and her husb...,0
10,AITAH for getting my Nieces birthday gift at a...,I am a full time university student that has d...,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 547 entries, 0 to 573
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      547 non-null    object
 1   selftext   547 non-null    object
 2   aitah_tag  547 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 17.1+ KB


## **Text Processing**

In [51]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [52]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
# Gets a list of english stop words
stop_words = stopwords.words('english')

In [54]:
def preprocess_data(reddit_post):
    reddit_post = str(reddit_post).lower()
    reddit_post = re.sub("[^a-zA-Z0-9\s]",'', reddit_post)

    temp_final =[]

    for word in reddit_post.split():
        if word =='' or '\r\n' in word or word in stop_words:
            None
        else:
            temp_final.append(word)

    return word_tokenize(' '.join(temp_final))

In [55]:
df['processed_selftext'] = df['selftext'].apply(preprocess_data)

In [56]:
df.head()

,title,selftext,aitah_tag,processed_selftext
0,AITA for cutting off a friend for being distant?,"\n\nso ill set the scene, i, (16M) and my two ...",0,"[ill, set, scene, 16m, two, friends, 16f, 17f,..."
1,AITAH for being pissed at my mom,So I (14M) have been playing guitar for 5ish y...,0,"[14m, playing, guitar, 5ish, years, passion, l..."
2,AITAH for making a girl cry at work,I work in the legal department at a corporate ...,0,"[work, legal, department, corporate, job, girl..."
3,AIT for telling my parents something my friend...,I've told this story in a few other subreddits...,0,"[ive, told, story, subreddits, im, gon, na, te..."
5,AITAH for asking my friend to do some work for...,Im an artist (30/m) and im currently working h...,0,"[im, artist, 30m, im, currently, working, hard..."


In [19]:
df['aitah_tag'].value_counts()

,count
aitah_tag,
0,501
1,46


## **Model Development**

### **Count Vectorizer**

In [ ]:
#Importing count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
#Creating a CountVectorizer model
df['selftext'] = df['processed_selftext']
bow_transformer= CountVectorizer(analyzer=preprocess_data).fit(df['selftext'])

In [22]:
#Print number of vocabulary/words
print(len(bow_transformer.vocabulary_))

10526


In [ ]:
article_bow=bow_transformer.transform(df['selftext'])

### **TF-IDF**

In [24]:
#Import TFIDF
from sklearn.feature_extraction.text import TfidfTransformer
#Making an instance of this transformer
tfidf_transformer=TfidfTransformer().fit(article_bow)

In [26]:
#Checking the tfidf of a particular word('husband'), it'll directly error if it doesnt see the word
tfidf_transformer.idf_[bow_transformer.vocabulary_['husband']]

2.8995560396837625

In [27]:
#Convert the entire bag of words corpus into a tfidf corpus at once
article_tfidf=tfidf_transformer.transform(article_bow)

### **Support Vector Machine**

In [ ]:
#lets import SVM
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [32]:
#think of the pipeline as the steps or methods on a task
text_clf_svm = Pipeline([('vect', CountVectorizer(analyzer=preprocess_data)),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='log_loss', penalty='elasticnet',
                                            alpha=0.001, random_state=42, class_weight='balanced')),
 ])

In [33]:
_=text_clf_svm.fit(article_train,category_train)

## **Model Evaluation**

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
prediction=text_clf_svm.predict(article_test)

In [35]:
# Check the class distribution in the training set
print("Training set class distribution:")
print(pd.Series(category_train).value_counts())

# Check the class distribution in the test set
print("Test set class distribution:")
print(pd.Series(category_test).value_counts())


Training set class distribution:
aitah_tag
0    348
1     34
Name: count, dtype: int64
Test set class distribution:
aitah_tag
0    153
1     12
Name: count, dtype: int64


In [36]:
# Generate a classification report
report = classification_report(category_test, prediction, zero_division=0)

In [37]:
print(df['aitah_tag'].value_counts())

aitah_tag
0    501
1     46
Name: count, dtype: int64


In [53]:
print(report)

              precision    recall  f1-score   support

           0       0.93      0.99      0.96       132
           1       0.00      0.00      0.00        10

    accuracy                           0.92       142
   macro avg       0.46      0.50      0.48       142
weighted avg       0.86      0.92      0.89       142



## **Testing**

In [41]:
# Example with manual mapping
label_mapping = {0: "NTA", 1: "YTA"}

# Predict the label
prediction_numeric = text_clf_svm.predict(["I gave her flowers"])[0]
prediction_label = label_mapping[prediction_numeric]

print(f"Predicted class: {prediction_label}")

Predicted class: NTA
